In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [5]:
def write_ROI(metric):
    bed_1 = pd.read_csv(f"CHG034730/outputs/CHG034730_{metric}_lowest_scores.bed", sep="\t", header=None)
    bed_2 = pd.read_csv(f"CHG034731/outputs/CHG034731_{metric}_lowest_scores.bed", sep="\t", header=None)
    metrics = {"Cosine_Similarity":5,"Dot_Product":6}
    with open(f"ROI\roi_{metric}.bed","w+") as f:
        f.write(f"Chromosome\tPos_6mer\tIndex_1\t{metric}_1\tIndex_2\t{metric}_2\n")
        for indexA, rowA in tqdm(bed_1.iterrows(),total=bed_1.shape[0]):
            for indexL, rowL in bed_2.iterrows():
                if rowA[0] == rowL[0] and abs(rowA[4]-rowL[4])<1000:
                    f.write(f"{rowA[0]}\t{rowA[4]}\t{indexA}\t{rowA[metrics[metric]]}\t{indexL}\t{rowL[metrics[metric]]}\n")        
    f.close()

In [3]:
def read_ROI(metric):
    roi_df = pd.read_csv(f"ROI_{metric}.bed",sep="\t",header=0)
    roi_df["Avg_Index"]=(roi_df["Index_1"]+roi_df["Index_2"])/2
    roi_df = roi_df.sort_values(by="Avg_Index").reset_index(drop=True)
    #roi_df = roi_df[(roi_df[f"{metric}_1"]+roi_df[f"{metric}_2"])/2<-0.10]
    return roi_df

In [4]:
#write_ROI("Cosine_Similarity")

100%|██████████| 1000/1000 [00:48<00:00, 20.65it/s]


In [5]:
#write_ROI("Dot_Product")

100%|██████████| 1000/1000 [00:52<00:00, 18.97it/s]


In [9]:
roi_cos = read_ROI("Cosine_Similarity")
roi_dot = read_ROI("Dot_Product")

In [1]:
display(roi_cos)
display(roi_dot)

NameError: name 'roi_cos' is not defined

In [12]:
#Check how many low dot products do not have low cosine similarities and vice versa
indices = np.empty((0,2))
for indexC, rowC in tqdm(roi_cos.iterrows(),total=roi_cos.shape[0]):
    for indexD, rowD in roi_dot.iterrows():
        if rowC["Pos_6mer"]==rowD["Pos_6mer"]:
            indices = np.vstack((indices, np.array([indexC,indexD])))
print(indices.shape)
print("cosine similarities not among the lowest dot products: ",np.setdiff1d(np.arange(0,roi_cos.shape[0]),indices[:,0]))
print("dot products not among the lowest cosine similarities: ",np.setdiff1d(np.arange(0,roi_cos.shape[0]),indices[:,1])) #In both cases: few entries, none of the very lowest -> all good

100%|██████████| 98/98 [00:00<00:00, 179.80it/s]

(90, 2)
cosine similarities not among the lowest dot products:  [53 75 79 81 88 89 95 96]
dot products not among the lowest cosine similarities:  [59 76 78 80 86 88 90 95 96]
